In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import torch.nn.init

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [8]:
# hyperparameters

learning_rate = 1e-3
nb_epochs = 20
batch_size = 100

In [9]:
# MINST datasets

mnist_train = datasets.MNIST(root = 'MNIST_data/',
                             train = True,
                             transform = transforms.ToTensor(),
                             download = True)

mnist_test = datasets.MNIST(root = 'MNIST_data/',
                            train = False,
                            transform = transforms.ToTensor(),
                            download = True)

In [10]:
train_data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                                batch_size = batch_size,
                                                shuffle = True,
                                                drop_last = True)

In [32]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding =1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3 * 3 * 128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias = True)
        
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out

In [33]:
model = CNN().to(device)

In [34]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)

In [35]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [36]:
# training
total_batch = len(train_data_loader)

for epoch in range(nb_epochs):
    avg_cost = 0
    
    for batch_idx, samples in enumerate(train_data_loader):
        X_train, Y_train = samples
        X_train = X_train.to(device)
        Y_train = Y_train.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X_train)
        
        cost = criterion(hypothesis, Y_train)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print("Epoch: {}/{} | Cost: {:.6f}".format(
        epoch + 1, nb_epochs,
        avg_cost))

Epoch: 1/20 | batch_size: 600/600 | Cost: 0.165752
Epoch: 2/20 | batch_size: 600/600 | Cost: 0.041669
Epoch: 3/20 | batch_size: 600/600 | Cost: 0.028845
Epoch: 4/20 | batch_size: 600/600 | Cost: 0.022941
Epoch: 5/20 | batch_size: 600/600 | Cost: 0.017330
Epoch: 6/20 | batch_size: 600/600 | Cost: 0.014145
Epoch: 7/20 | batch_size: 600/600 | Cost: 0.012389
Epoch: 8/20 | batch_size: 600/600 | Cost: 0.009976
Epoch: 9/20 | batch_size: 600/600 | Cost: 0.009409
Epoch: 10/20 | batch_size: 600/600 | Cost: 0.008536
Epoch: 11/20 | batch_size: 600/600 | Cost: 0.006862
Epoch: 12/20 | batch_size: 600/600 | Cost: 0.008533
Epoch: 13/20 | batch_size: 600/600 | Cost: 0.007545
Epoch: 14/20 | batch_size: 600/600 | Cost: 0.003427
Epoch: 15/20 | batch_size: 600/600 | Cost: 0.005114
Epoch: 16/20 | batch_size: 600/600 | Cost: 0.006011
Epoch: 17/20 | batch_size: 600/600 | Cost: 0.005023
Epoch: 18/20 | batch_size: 600/600 | Cost: 0.005941
Epoch: 19/20 | batch_size: 600/600 | Cost: 0.003834
Epoch: 20/20 | batch_

In [ ]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:", accuracy.item())